In [ ]:
import geocoder
import ephem
import ephem.stars
import datetime
import math
import cmath
import sympy
import functools
import numpy as np
import qutip
import random
import vpython
import mpmath

# Stereographically projects from complex plane + infinity to XYZ on sphere
def scalar_to_vector(z):
    if z == float('inf'):
        return [0,0,1]
    x = z.real
    y = z.imag
    return [(2*x)/(1.+(x**2)+(y**2)), \
           (2*y)/(1.+(x**2)+(y**2)), \
           (-1.+(x**2)+(y**2))/(1.+(x**2)+(y**2))]

# Stereographically projects from XYZ on sphere to complex plane + infinity
def vector_to_scalar(x, y, z):
    if z == 1:
        return float('inf') 
    else:
        return complex(x/(1-z), y/(1-z))

# Gets XYZ of Majorana stars for given vector
def constellate(v):
    polynomial = v.T.tolist()
    polynomial = [(((-1)**i) * math.sqrt(combos(len(polynomial)-1,i))) * polynomial[i] for i in range(len(polynomial))]
    try:
        roots = [complex(root) for root in mpmath.polyroots(polynomial)]
    except:
        print(polynomial)
    return [scalar_to_vector(root) for root in roots] 

# Normalize vector
def normalize(v):
    norm = np.linalg.norm(v)
    if norm == 0: 
       return v
    return v / norm

# Scalar product between two matrices
def scalar_product(m, n):
    return 0.5*np.trace(np.dot(np.conjugate(m).T, n))

# Decompose 2x2 matrix into tI + xX + yY + zZ, 
# returning [t,x,y,z] :|
def pauli_decompose(m):
    t = scalar_product(np.eye(2), m).real
    x = scalar_product(qutip.sigmax().full(), m).real
    y = scalar_product(qutip.sigmay().full(), m).real
    z = scalar_product(qutip.sigmaz().full(), m).real
    return [t/math.sqrt(t**2+x**2+y**2+z**2), x/t, y/t, z/t]

# (a b)
def combos(a,b):
    f = math.factorial
    return f(a) / f(b) / f(a-b)

star_names = [star.split(",")[0] for star in ephem.stars.db.split("\n")][:-1]
# Get current location and time on earth
location = geocoder.google('476 Jefferson Street, Brooklyn')
latitude, longitude = location.latlng

# Given current location and date, 
# returns XYZ coordinates of the 7 planets
# and TXYZ coordinates of the 3 sublunary stars
# and the current time and the eigenstars
def cycle(t):
	global latitude
	global longitude
	global star_names
	now = datetime.datetime.now() + datetime.timedelta(minutes=t)
	#now = datetime.datetime.strptime('Jan 21 2018  7:13AM', '%b %d %Y %I:%M%p') + datetime.timedelta(minutes=t)
	# Get empheres for the 7 planets
	observer = ephem.Observer()
	observer.lat = latitude
	observer.lon = longitude
	observer.date = now
	planets = [ephem.Sun(observer), ephem.Moon(observer), ephem.Mercury(observer),\
			ephem.Venus(observer), ephem.Mars(observer), ephem.Jupiter(observer),\
			ephem.Saturn(observer)]
	fixed_stars = [ephem.star(star_name, observer) for star_name in star_names]
	fixed_stars_XYZ = []
	for fixed_star in fixed_stars:
		altitude = fixed_star.alt
		azimuth = fixed_star.az
		x = math.sin(azimuth)*math.cos(altitude)
		y = math.cos(altitude)*math.cos(azimuth)
		z = math.sin(altitude)
		fixed_stars_XYZ.append([x,y,z])
	# Convert alt/azi -> xyz -> X+Yi
	roots = []
	permutation_symmetric_XYZ = []
	for planet in planets:
		altitude = planet.alt
		azimuth = planet.az
		x = math.sin(azimuth)*math.cos(altitude)
		y = math.cos(altitude)*math.cos(azimuth)
		z = math.sin(altitude)
		#right_ascension = planet.ra
		#declination = planet.dec
		#x = math.cos(right_ascension)*math.cos(declination)
		#y = math.cos(right_ascension)*math.sin(declination)
		#z = math.sin(right_ascension)
		permutation_symmetric_XYZ.append([x,y,z])
		X = float('inf') 
		Y = float('inf') 
		if z != 1:
			X = x/(1-z)
			Y = y/(1-z)
		root = complex(X, Y)
		r, theta = cmath.polar(root)
		roots.append(root)
	# Construct polynomial given roots
	s = sympy.symbols("s")
	polynomial = sympy.Poly(functools.reduce(lambda a, b: a*b, [s+root for root in roots]), domain="CC")
	# Extract coefficients
	coefficients = polynomial.coeffs()
	coordinates = [coefficients[i]/(((-1)**i) * math.sqrt(combos(len(coefficients)-1,i))) for i in range(len(coefficients))]
	# Convert to quantum state of proper dimensionality (3 distinguishable qubits)
	state = qutip.Qobj(np.array(coordinates).T)
	state.dims = [[2,2,2],[1,1,1]]
	# Decompose the total pure state by partial tracing over the complements of each 2x2 subsystem in turn
	distinguishable_qubits = [state.ptrace(i) for i in range(3)]
	# Decompose the resulting density matrices in the Pauli basis, giving TXYZ coordinates
	distinguishable_TXYZ = [pauli_decompose(qubit.full()) for qubit in distinguishable_qubits]
    # Get eigenstars of the density matrices
	eig_TXYZ = []
	for qubit in distinguishable_qubits:
		L, V = np.linalg.eig(qubit.full())
		for i in range(len(V)):
			eig_TXYZ.append(constellate(np.conjugate(V[i]))[0])
	return permutation_symmetric_XYZ, distinguishable_TXYZ, now, eig_TXYZ, fixed_stars_XYZ

# Vpython init
vpython.scene.width = 1000
vpython.scene.height = 1000
vpython.scene.range = 1.5
vpython.scene.forward = vpython.vector(0, 1, 0)
vpython.scene.up = vpython.vector(-1, 0, 0)

# Reference sphere
vsphere = vpython.sphere(pos=vpython.vector(0,0,0), radius=1, color=vpython.color.blue, opacity=0.5)

# Reference points
cardinal_rose = [vpython.text(text="W", pos=vpython.vector(0,0,1), align="center", height=0.1, twosided=False),\
                 vpython.text(text="E", pos=vpython.vector(0,0,-1), align="center", height=0.1, twosided=False),\
                 vpython.text(text="S", pos=vpython.vector(0,-1,0), align="center", height=0.1, twosided=False),\
                 vpython.text(text="N", pos=vpython.vector(0,1,0), align="center", height=0.1, twosided=False),\
                 vpython.text(text="0", pos=vpython.vector(1,0,0), align="center", height=0.1, twosided=False),\
                 vpython.text(text="1", pos=vpython.vector(-1,0,0), align="center", height=0.1, twosided=False)]

# Create 3d objects with initial state
t = 0
above, below, now, eigs, fixed_stars = cycle(t)
stamp = vpython.label(pos=vpython.vector(0,0,0), text=str(now), height=10)
vabove = [vpython.sphere(pos=vpython.vector(*above[0]), color=vpython.color.yellow, radius=0.1, emissive=True, make_trail=False),\
		  vpython.sphere(pos=vpython.vector(*above[1]), color=vpython.color.white, radius=0.1, emissive=True, make_trail=False),\
		  vpython.sphere(pos=vpython.vector(*above[2]), color=vpython.color.blue, radius=0.1, emissive=True, make_trail=False),\
		  vpython.sphere(pos=vpython.vector(*above[3]), color=vpython.color.green, radius=0.1, emissive=True, make_trail=False),\
		  vpython.sphere(pos=vpython.vector(*above[4]), color=vpython.color.red, radius=0.1, emissive=True, make_trail=False),\
		  vpython.sphere(pos=vpython.vector(*above[5]), color=vpython.color.orange, radius=0.1, emissive=True, make_trail=False),\
		  vpython.sphere(pos=vpython.vector(*above[6]), color=vpython.color.gray(0.5), radius=0.1, emissive=True, make_trail=False)]
vbelow = [vpython.sphere(pos=vpython.vector(*star[1:]),\
			color=vpython.color.hsv_to_rgb(vpython.vector(float(star[0]),1,1)), radius=0.1, emissive=True,\
					make_trail=True)\
				for star in below]
vbelow[0].trail_color = vpython.color.red
vbelow[1].trail_color = vpython.color.green
vbelow[2].trail_color = vpython.color.blue

veigs = [vpython.sphere(pos=vpython.vector(*star), color=vpython.color.black, radius=0.05, emissive=True, make_trail=False)\
				for star in eigs]
vlines = [vpython.curve(pos=[vpython.vector(*eigs[i]), vpython.vector(*eigs[i+1])]) for i in range(0, len(eigs), 2)]
vfixed_stars = [vpython.sphere(pos=vpython.vector(*star), color=vpython.color.white, radius=0.01, emissive=True, make_trail=False)\
				for star in fixed_stars]
vfixed_stars[star_names.index("Sirius")].color = vpython.color.red
vfixed_stars[star_names.index("Betelgeuse")].color = vpython.color.yellow
vfixed_stars[star_names.index("Rigel")].color = vpython.color.yellow
vfixed_stars[star_names.index("Bellatrix")].color = vpython.color.yellow
vfixed_stars[star_names.index("Mintaka")].color = vpython.color.yellow
vfixed_stars[star_names.index("Alnilam")].color = vpython.color.yellow
vfixed_stars[star_names.index("Alnitak")].color = vpython.color.yellow
vfixed_stars[star_names.index("Saiph")].color = vpython.color.yellow
vfixed_stars[star_names.index("Polaris")].color = vpython.color.blue

t += 1

# Cycle -> Redraw
while True:
	vpython.rate(300)
	above, below, now, eigs, fixed_stars = cycle(t)
	for i in range(len(above)):
		vabove[i].pos = vpython.vector(*above[i])
	for i in range(len(below)):
		vbelow[i].color = vpython.color.hsv_to_rgb(vpython.vector(float(below[i][0]),1,1))
		vbelow[i].pos = vpython.vector(*below[i][1:])
	for i in range(len(eigs)):
		veigs[i].pos = vpython.vector(*eigs[i])
	for i in range(len(vlines)):
		vlines[i].modify(0, pos=vpython.vector(*eigs[2*i]))
		vlines[i].modify(1, pos=vpython.vector(*eigs[2*i+1]))
	for i in range(len(fixed_stars)):
		vfixed_stars[i].pos = vpython.vector(*fixed_stars[i])
	stamp.text = str(now)
	t += 15

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>